In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
imgsize = (150, 150)
bsize = 32
epochs = 10
train_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)  
train_generator = train_data.flow_from_directory(
    'datas/',
    target_size=imgsize,
    batch_size=bsize,
    class_mode='categorical',
    subset='training')
validation_generator = train_data.flow_from_directory(
    'datas/',
    target_size=imgsize,
    batch_size=bsize,
    class_mode='categorical',
    subset='validation')
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    train_generator,
    steps_per_epoch=10,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=10)
model.save('weed.h5')


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('weed_detection_model.h5')
cap = cv2.VideoCapture(0)  
while True:
    ret, frame = cap.read()
    if not ret:
        break
    img = cv2.resize(frame, (150, 150))
    img = img.astype('float32')
    img /= 255
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    pred_class = np.argmax(prediction, axis=1)[0]
    cv2.putText(frame, f'Predicted Weed: {pred_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Weed Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()